In [2]:
!pip install pandas bert_score scipy

In [3]:
import pandas as pd
from bert_score import score
from scipy.stats import pearsonr

In [12]:
# Read CSV file into a pandas DataFrame
file_path = 'test_stata.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

In [14]:
# Calculate BERT score
ref_sentences = df['linearized_input'].tolist()
hyp_sentences = df['output'].tolist()
P, R, F1 = score(hyp_sentences, ref_sentences, model_type='bert-base-multilingual-cased', device="cuda")  # Change 'en' to the appropriate language code if needed

# Create a new column for F1 scores
df['bert_score_f1'] = F1.tolist()

# Calculate Pearson correlation
pearson_corr, _ = pearsonr(df['bert_score_f1'], df['attributable'])

# Print the Pearson correlation
print(f"Pearson correlation between BERT score F1 and attributable: {pearson_corr}")

Pearson correlation between BERT score F1 and attributable: 0.12296068878292361


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

summary = "An elephant threw a stone at a girl."
article = "The stone got past the elephant's fence and a ditch separating the animal and visitors, the zoo said in a statement.The girl was taken to hospital and died within a few hours, the zoo added.The zoo statement said the enclosure met international standards and said 'this kind of accident is rare, unpredictable and unusual'.Africa Live: More on this and other storiesThe statement went on (in French) to point out two other recent incidents in the US:Phyllis Lee, Scientific Director of the Amboseli Trust for Elephants, says that targeted throwing of stones and branches by elephants is very unusual.'It can happen when elephants are frustrated or bored. In my opinion, it's unlikely the elephant was directly targeting the girl - but exhibiting frustration. You can't predict what animals in captivity will do.'The moments after the girl was struck at Rabat Zoo on Tuesday were filmed by a bystander and uploaded onto YouTube.The video shows the elephant waving its trunk behind a fence and swerves round to show a stone on the ground.Metres away people are gathered around the girl, holding her head and stroking her leg."
input = [[summary, article]]

tokenizer = AutoTokenizer.from_pretrained("roberta-base", padding="max_length", truncation=True)
factkb = AutoModelForSequenceClassification.from_pretrained("bunsenfeng/FactKB", num_labels = 2)

tokens = tokenizer(input, return_tensors="pt", padding="max_length", truncation=True)
result = torch.softmax(factkb(**tokens).logits, dim = 1)

print('The factuality score (0-1, 1 as factual) is: ', float(result[0][1]))

The factuality score (0-1, 1 as factual) is:  0.023097408935427666
